<a href="https://colab.research.google.com/github/tiago15anjos/DataSat_Unificado_Final/blob/main/Dataset_Unificado_ICJ%E2%80%AF%2B%E2%80%AFEduca%C3%A7%C3%A3o_(2010%E2%80%912023).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

# ─── 1. Arquivos no /content ─────────────────────────────────────────────
ICJ_FILE  = "ICJ_Municipio_Ano_2010_2023.xlsx"
EB_FILE   = "MATRICULAS_EB_ENSF_ENSM_ENSMT_2010_2023_LIMPO.xlsx"
FORM_FILE = "FORMACAO_PROF_EB_2010_2023_LIMPO.xlsx"
SUP_FILE  = "MATRICULA_ENS_SUP_2010_a_2023_LIMPO.xlsx"

# ─── 2. Listas de colunas ────────────────────────────────────────────────
DIM_COMUNS = [
    "Regiao_Geografica", "Unidade_Federacao", "Nome_Municipio",
    "Codigo_IBGE", "Ano"
]
DIM_EXTRA = ["Sigla_UF", "Pais"]

ICJ_METRICS = [
    "Qtd_Bolsistas",
    "Valor_Rs_mean", "Valor_Rs_sum",
    "Valor_USS_mean", "Valor_USS_sum",
    "Benef_Modal/Ano_mean", "Benef_Modal/Ano_sum",
    "Bolsa/Ano_mean", "Bolsa/Ano_sum",
    "Auxílio/Ano_mean", "Auxílio/Ano_sum"
]
EB_METRICS   = ["Ens_Fund_Anos_Finais", "Ensino_Medio", "Ensino_Medio_Tecnico"]
FORM_METRICS = ["Graduação_Total", "Com_Especialização", "Com_Mestrado", "Com_Doutorado"]
SUP_METRICS  = [
    "Total_Geral", "Total_Publica", "Federal", "Estadual", "Municipal",
    "Total_Privada", "Com Fins Lucrativos", "Sem Fins Lucrativos"
]

# ─── 3. Ler planilhas (mantendo dimensões completas) ─────────────────────
icj  = pd.read_excel(ICJ_FILE ,
                     usecols=lambda c: c in DIM_COMUNS + DIM_EXTRA + ICJ_METRICS)

# → Matrículas EB: renomear 'Unidade_da_Federacao'
eb   = pd.read_excel(EB_FILE)
eb   = eb.rename(columns={"Unidade_da_Federacao": "Unidade_Federacao"})
eb   = eb[DIM_COMUNS + EB_METRICS]     # mantém apenas o que interessa

form = pd.read_excel(FORM_FILE)
form = form[DIM_COMUNS + FORM_METRICS]

sup  = pd.read_excel(SUP_FILE)
sup  = sup[DIM_COMUNS + SUP_METRICS]

# ─── 4. Mesclar EB com ICJ (mantendo campos duplicados) ──────────────────
merged = icj.merge(
    eb, on=["Codigo_IBGE", "Ano"], how="outer", suffixes=("", "_eb")
)

# Preencher dimensões faltantes com valores que vieram do EB
for col in DIM_COMUNS:
    if col+"_eb" in merged.columns:
        merged[col] = merged[col].combine_first(merged[col+"_eb"])
        merged.drop(columns=col+"_eb", inplace=True)

# ─── 5. Juntar FORM (com preenchimento) ──────────────────────────────────
merged = merged.merge(form, on=["Codigo_IBGE", "Ano"], how="outer", suffixes=("", "_form"))
for col in DIM_COMUNS:
    if col+"_form" in merged.columns:
        merged[col] = merged[col].combine_first(merged[col+"_form"])
        merged.drop(columns=col+"_form", inplace=True)

# ─── 6. Juntar SUP (com preenchimento) ───────────────────────────────────
merged = merged.merge(sup, on=["Codigo_IBGE", "Ano"], how="outer", suffixes=("", "_sup"))
for col in DIM_COMUNS:
    if col+"_sup" in merged.columns:
        merged[col] = merged[col].combine_first(merged[col+"_sup"])
        merged.drop(columns=col+"_sup", inplace=True)

# ─── 7. Reordenar colunas ────────────────────────────────────────────────
ORDER = (
    DIM_COMUNS + DIM_EXTRA +
    ICJ_METRICS +
    EB_METRICS +
    FORM_METRICS +
    SUP_METRICS
)
ORDER = [c for c in ORDER if c in merged.columns]  # garante existência
merged = merged[ORDER]

# ─── 8. Exportar resultado ───────────────────────────────────────────────
OUTPUT = "ICJ_EB_SUP_FORM_Merged_2010_2023.xlsx"
merged.to_excel(OUTPUT, index=False)
print("✅ Arquivo final salvo como:", OUTPUT)


✅ Arquivo final salvo como: ICJ_EB_SUP_FORM_Merged_2010_2023.xlsx
